In [4]:
import pandas as pd
import json
from rapidfuzz import process, fuzz
from ai_integration.chatgpt_kpi_assistant import ai_fallback_column_mapping


pd.set_option('display.max_columns', None)


class DataHandler:
    def __init__(self, source_type, source, mapping_path):
        """
        :param source_type: Type of data source (excel, csv, db, api, json).
        :param source: File path, database connection, or API endpoint.
        :param mapping_path: Path to JSON column mapping file.

        """
        self.source_type = source_type
        self.source = source
        self.mapping_path = mapping_path
        self.column_mappings = self._load_column_mapping()
        self.data = None
        

    def load_data(self):
        """Load data based on the selected source type and apply column mapping."""
        try:
            if self.source_type == "excel":
                self.data = pd.read_excel(self.source)
            elif self.source_type == "csv":
                self.data = pd.read_csv(self.source)
            elif self.source_type == "db":
                self.data = self._load_from_db()
            elif self.source_type == "api":
                self.data = self._load_from_api()
            elif self.source_type == "json":
                self.data = self._load_from_json()
            else:
                return f"Unsupported source type: {self.source_type}"

            # Apply column mapping after loading data
            self._apply_column_mapping()

            return self.data
        except Exception as e:
            return f"Error loading data: {str(e)}"

    def _load_from_db(self):
        """Load data from an SQLite database (can be extended for other DBs)."""
        conn = sqlite3.connect(self.source)  # Example for SQLite
        query = "SELECT * FROM kpi_data"  # Modify as needed
        return pd.read_sql(query, conn)

    def _load_from_api(self):
        """Fetch data from an API endpoint."""
        response = requests.get(self.source)
        if response.status_code == 200:
            return pd.DataFrame(response.json())  # Assuming API returns JSON
        return f"API request failed with status: {response.status_code}"

    def _load_from_json(self):
        """Load data from a JSON file."""
        with open(self.source, "r") as f:
            return pd.DataFrame(json.load(f))
    
    def _load_column_mapping(self):
        """Loads column mapping definitions from a JSON file."""
        try:
            with open(self.mapping_path, "r", encoding="utf-8") as file:
                return json.load(file)
        except Exception as e:
            print(f"Error loading column mapping: {e}")
            return {}

    def _find_best_match(self, detected_column):
            """Finds the best match for a detected column using RapidFuzz."""
            best_match = None
            highest_score = 0
            for standard_name, details in self.column_mappings.items():
                possible_matches = [standard_name] + details["alternative"]
                match, score,index = process.extractOne(detected_column, possible_matches, scorer=fuzz.ratio)

                if score > highest_score:
                    highest_score = score
                    best_match = standard_name
            # Return highest score
            if highest_score > 75:
                return best_match, highest_score
            else: 
                return None, None
        
    def _apply_column_mapping(self):
        """Maps detected columns to standarized names using RapidFuzz and AI fallback"""
        detected_columns = list(self.data.columns)
        mapped_columns = {}
        unmatched_columns = []
        
        # Step 1: Try RapidFuzz first
        for detected_col in detected_columns:
            best_match, highest_score  = self._find_best_match(detected_col)
            print(f"Input: {detected_col} -> Standard: {best_match} with a score of {highest_score}")

            if best_match:
                mapped_columns[detected_col] = best_match

            else:
                unmatched_columns.append(detected_col)

        # Step 2: Use ChatGPT for all unmatched columns in one request
        if unmatched_columns:
            ai_suggestions = ai_fallback_column_mapping(unmatched_columns, list(self.column_mappings.keys()))
            # ai_suggestions = [None for i in unmatched_columns] # Place holder while AI tool is addressed
            

            # Update mappings with AI suggestions
            for i, col in enumerate(unmatched_columns):
                mapped_columns[col] = ai_suggestions[i] if ai_suggestions[i] else col  # Keep original if AI fails

        # Step 3: Rename columns in the DataFrame
        self.data.rename(columns=mapped_columns, inplace=True)

        # Rename columns in the DataFrame
        self.data.rename(columns=mapped_columns, inplace=True)


After ChatGPT

In [5]:
mapping_path = "mappings/well_stimulation_mapping.json"
file_path = "Estimulaciones_edit.xlsx"
data_handler = DataHandler(source_type="excel", source=file_path, mapping_path=mapping_path)
data_handler.load_data()

Input: Región -> Standard: Region with a score of 100.0
Input: Activo  -> Standard: Lease with a score of 92.3076923076923
Input: Campo -> Standard: Field with a score of 100.0
Input: Pozo -> Standard: Well ID with a score of 100.0
Input: Latitud Conductor -> Standard: Latitude with a score of 100.0
Input: Longitud Conductor -> Standard: Longitude with a score of 100.0
Input: Pera -> Standard: Operational Permit with a score of 100.0
Input: Costo (USD) -> Standard: Cost (USD) with a score of 100.0
Input: Fecha -> Standard: Date with a score of 100.0
Input: Categoría del Pozo -> Standard: Well Category with a score of 100.0
Input: Intervención -> Standard: Intervention Type with a score of 100.0
Input: Cía -> Standard: Company with a score of 100.0
Input: Contrato -> Standard: Contract with a score of 100.0
Input: Solvente [m3] -> Standard: None with a score of None
Input: Acido [m3] -> Standard: None with a score of None
Input: DIVER. [m3] -> Standard: Divergent Volume (m3) with a scor

,Region,Lease,Field,Well ID,Latitude,Longitude,Operational Permit,Cost (USD),Date,Well Category,Intervention Type,Company,Contract,Solvent Volume (m3),Acid Volume (m3),Divergent Volume (m3),Inhibitor Volume (m3),Neutralizer Volume (m3),Linear Gel Volume (m3),Treated Water Volume (m3),Brine Volume (m3),Water Volume (m3),Liquid Displacement (m3),Nitrogen Displacement (m3),Total Fluid Volume (m3),Producing Formation,Cima [md],Base [md],Qo Before (bpd),Qo After (bpd),Qo Difference (bpd),Pwf Before (kg/cm²),Pwf After (kg/cm²),Pwf Difference (kg/cm²),Pwf Before (kg/cm²),Pwf After (kg/cm²),Sensor / Nodal,Pwf Difference (kg/cm²),Reaction Time in Bottom (hrs),Revenue (USD)
0,Sur,APSL,SAMARIA,SAMARIA 678,NaN,NaN,Pozo1,35920.8000,2020-09-07,DESARROLLO,LIMPIEZA CON TF,NS,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,10.0,NaN,215.0,82.0,6497.0,8498.0,2001.0,283.0,426.0,143.0,NaN,NaN,Nodal,0.0,NaN,65
1,Sur,APSL,SAMARIA,SAMARIA 695,NaN,NaN,Pozo2,35920.8000,2020-09-07,DESARROLLO,LIMPIEZA CON TF,NS,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,10.0,NaN,232.0,880.0,4600.0,6412.0,1812.0,430.0,456.0,26.0,600.0,756.0,Sensor,156.0,NaN,65
2,Sur,APSL,CUNDUACAN,CUNDUACAN 30A,NaN,NaN,Pozo3,32169.4269,2020-09-22,DESARROLLO,LIMPIEZA CON TF,NS,NaN,10.0,10.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,80.0,NaN,30.0,NaN,242.0,765.0,3000.0,5800.0,2800.0,300.0,470.0,170.0,520.0,710.0,Sensor,190.0,NaN,65
3,Sur,APSL,SAMARIA,SAMARIA 702,NaN,NaN,Pozo4,14831.8000,2020-10-30,DESARROLLO,LIMPIEZA CON TF,NS,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,10.0,NaN,303.0,906.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,Nodal,0.0,NaN,65
4,Sur,APSL,SINI,SINI 2,NaN,NaN,Pozo5,34867.7396,2020-11-18,DESARROLLO,LIMPIEZA CON TF,NS,NaN,20.0,3.0,NaN,1.5,1.5,NaN,NaN,NaN,NaN,68.0,NaN,26.0,NaN,133.0,1005.0,2800.0,4600.0,1800.0,400.0,432.0,32.0,610.0,650.0,Sensor,40.0,NaN,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Sur,APBJ,CUPACHE,CUPACHE 1,NaN,NaN,NaN,29663.6000,2022-03-25,DESARROLLO,LIMPIEZA CON TF,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65
194,Sur,APBJ,BELLOTA,BELLOTA 41,NaN,NaN,NaN,19093.7800,2022-04-05,DESARROLLO,LIMPIEZA CIRCULADA,NaN,NaN,10.0,3.0,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65
195,Sur,APBJ,TUPILCO,TUPILCO 208T,NaN,NaN,NaN,10128.7500,2022-04-05,DESARROLLO,LIMPIEZA CON TF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65
196,Sur,APBJ,BELLOTA,BELLOTA 116A,NaN,NaN,NaN,27867.8200,2022-04-22,DESARROLLO,LIMPIEZA CIRCULADA,NaN,NaN,14.0,5.0,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,21.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65


In [6]:
with open("mappings/test.json", 'r') as j:
     contents = json.loads(j.read())


In [7]:
contents


['Solvent Volume (m3)',
 'Acid Volume (m3)',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Solvent Volume (m3)',
 'Acid Volume (m3)',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]